In [1]:
import numpy as np
import cv2

In [2]:
cap = cv2.VideoCapture(0)

In [3]:
detector = cv2.CascadeClassifier("../haarcascade_frontalface_default.xml")

In [4]:
flag= True

while(flag):
    ret, frame = cap.read()
    if ret == True:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector.detectMultiScale(frame,1.3,5)
        if(len(faces)):
            x,y,w,h = faces[0]
            flag=False
            track_window = (x, y, w, h)

            # set up the ROI for tracking
            roi = frame[y:y+h, x:x+w]
            hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
            mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
            roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
            cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

            # Setup the termination criteria, either 10 iteration or move by at least 1 pt
            term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

            while(True):
                ret, frame = cap.read()

                if ret == True:
                    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
                    dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

                    # apply camshift to get the new location
                    ret, track_window = cv2.CamShift(dst, track_window, term_crit)

                    # Draw it on image
                    pts = cv2.boxPoints(ret)
                    pts = np.int0(pts)
                    img2 = cv2.polylines(frame,[pts],True, 255,2)
#                     img = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,255),2)
                    cv2.imshow('Frame',img2)

                    k = cv2.waitKey(30) & 0xff
                    if k == 27:
                        break
                else:
                    break
cap.release()
cv2.destroyAllWindows()